In [1]:
%%html
<audio src="https://oob.alwaysdata.net/silence.mp3" controls autoplay loop></audio>

In [2]:
#!git clone https://github.com/AnupJindal07/gemma-3-finetune-resume-data.git

Cloning into 'gemma-3-finetune-resume-data'...
remote: Enumerating objects: 141, done.
remote: Counting objects: 100% (141/141), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 141 (delta 67), reused 99 (delta 38), pack-reused 0 (from 0)
Receiving objects: 100% (141/141), 978.07 KiB | 12.70 MiB/s, done.
Resolving deltas: 100% (67/67), done.


In [3]:
!pip install -q torch transformers sentence-transformers datasets peft accelerate bitsandbytes bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 45.2 MB/s eta 0:00:00


In [4]:
%cd /content/gemma-3-finetune-resume-data
!pwd

/content/gemma-3-finetune-resume-data
/content/gemma-3-finetune-resume-data


In [5]:
#!mkdir -p data/raw_data data/processed_data models/checkpoints

In [6]:
from datetime import datetime

from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments, BitsAndBytesConfig, DataCollatorForLanguageModeling
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

In [7]:
print("CUDA available:", torch.cuda.is_available())

CUDA available: True


In [8]:
model_name = "google/gemma-3-4b-it"
model_checkpoint_path = "models/checkpoints"
fine_tuned_model_name = "fine_tuned_gemma-3-4b-it_resume"
train_version = datetime.now().strftime("%Y%m%d-%H%M%S")
trained_model_path = f"models/{fine_tuned_model_name}_{train_version}"

dataset_file_path = "data/datasets/dataset_01.csv"

process_file_path = "data/processed_data/resume_training_data_01.jsonl"

In [9]:
model_name = model_name

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Use the EOS token as padding
tokenizer.padding_side = "right"  # Ensure padding is on the right for causal LM

# Apply 4-bit quantization
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [10]:
print("Loading model...")

Loading model...


In [11]:
# Load model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto",
    dtype=torch.float16,
    quantization_config=quantization_config,
)

# Enable training mode
model.config.use_cache = False

# Prepare model for k-bit training BEFORE applying LoRA
model = prepare_model_for_kbit_training(model)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

In [12]:
config = LoraConfig(
    task_type="CAUSAL_LM",          # Causal language modeling
    r=8,                                   # Rank of adaptation
    lora_alpha=16,                         # LoRA scaling parameter
    lora_dropout=0.05,                      # LoRA dropout
    target_modules=[                       # Target modules for Gemma-3
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ],
    bias="none",                           # No bias training
    inference_mode=False                   # Training mode
)

model = get_peft_model(model, config)

# Print LoRA info to verify it's working
model.print_trainable_parameters()

trainable params: 16,394,240 || all params: 4,316,473,712 || trainable%: 0.3798


In [13]:
print(model)

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Gemma3ForConditionalGeneration(
      (model): Gemma3Model(
        (vision_tower): SiglipVisionModel(
          (vision_model): SiglipVisionTransformer(
            (embeddings): SiglipVisionEmbeddings(
              (patch_embedding): Conv2d(3, 1152, kernel_size=(14, 14), stride=(14, 14), padding=valid)
              (position_embedding): Embedding(4096, 1152)
            )
            (encoder): SiglipEncoder(
              (layers): ModuleList(
                (0-26): 27 x SiglipEncoderLayer(
                  (layer_norm1): LayerNorm((1152,), eps=1e-06, elementwise_affine=True)
                  (self_attn): SiglipAttention(
                    (k_proj): lora.Linear4bit(
                      (base_layer): Linear4bit(in_features=1152, out_features=1152, bias=True)
                      (lora_dropout): ModuleDict(
                        (default): Dropout(p=0.05, inplace=False)
                      )
                  

In [14]:
dataset = load_dataset("json", data_files={"train": process_file_path})

Generating train split: 0 examples [00:00, ? examples/s]

In [15]:
def tokenize_function(examples):
    input_texts = examples["prompt"]
    output_texts = examples["response"]

    # Format conversations using Gemma-3 IT chat template
    conversations = []
    for prompt, response in zip(input_texts, output_texts):
        # Create conversation in Gemma-3 IT format
        conversation = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": response}
        ]
        conversations.append(conversation)

    # Use the tokenizer's chat template to format the conversations
    formatted_texts = []
    for conversation in conversations:
        # Apply chat template
        formatted_text = tokenizer.apply_chat_template(
            conversation,
            tokenize=False,
            add_generation_prompt=False
        )
        formatted_texts.append(formatted_text)

    # Tokenize the formatted conversations
    tokenized = tokenizer(
        formatted_texts,
        padding="max_length",
        truncation=True,
        max_length=2048,
        return_tensors=None
    )

    # Create labels by masking the user prompt tokens
    labels = []
    for i, conversation in enumerate(conversations):
        # Create the user part only to find where assistant response starts
        user_conversation = [{"role": "user", "content": conversation[0]["content"]}]
        user_text = tokenizer.apply_chat_template(
            user_conversation,
            tokenize=False,
            add_generation_prompt=True  # This adds the assistant prompt
        )

        # Tokenize user part to find the boundary
        user_tokens = tokenizer(user_text, add_special_tokens=False)["input_ids"]
        full_tokens = tokenized["input_ids"][i]

        # Create label sequence - start with all masked
        label_seq = [-100] * len(full_tokens)

        # Only learn from the assistant response part
        user_length = len(user_tokens)
        if user_length < len(full_tokens):
            # Copy the assistant response tokens to labels (unmask them)
            for j in range(user_length, len(full_tokens)):
                if full_tokens[j] != tokenizer.pad_token_id:  # Don't learn from padding
                    label_seq[j] = full_tokens[j]

        labels.append(label_seq)

    # Debug: print first example
    if len(labels) > 0:
        non_masked_count = sum(1 for x in labels[0] if x != -100)
        print(f"Debug: First example has {non_masked_count} non-masked tokens to learn from")

    tokenized["labels"] = labels
    return tokenized


In [16]:
# Apply tokenization
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Remove the original text columns that are causing issues
tokenized_datasets = tokenized_datasets.remove_columns(["prompt", "response"])

# Split into train and eval datasets (85-15 ratio)
train_eval_split = tokenized_datasets["train"].train_test_split(test_size=0.15, seed=42)
train_dataset = train_eval_split["train"]
eval_dataset = train_eval_split["test"]

print(f"Train dataset size: {len(train_dataset)}")
print(f"Eval dataset size: {len(eval_dataset)}")

Map:   0%|          | 0/2999 [00:00<?, ? examples/s]

Debug: First example has 143 non-masked tokens to learn from
Debug: First example has 142 non-masked tokens to learn from
Debug: First example has 149 non-masked tokens to learn from
Train dataset size: 2549
Eval dataset size: 450


In [17]:
batch_size = 4
gradient_accumulation_steps = 8
num_epochs = 6

In [18]:
# Don't use DataCollatorForLanguageModeling as it interferes with our custom labels
# Use a simple data collator that just handles padding
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(
    tokenizer=tokenizer,
    return_tensors="pt"
)

In [19]:
training_args = TrainingArguments(
    output_dir=model_checkpoint_path,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size*2,
    gradient_accumulation_steps=gradient_accumulation_steps,
    num_train_epochs=num_epochs,
    learning_rate=1e-4,  # Higher learning rate for LoRA, 2e-4
    weight_decay=0.01,
    #warmup_steps=20,
    warmup_ratio=0.05,
    logging_dir="logs",
    logging_steps=25,

    eval_steps=50,
    eval_strategy="steps",
    save_steps=50,
    save_total_limit=4,
    load_best_model_at_end=True,
    max_grad_norm=1.0,   # Gradient clipping
    #metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=False,                      # Use bfloat16 instead
    bf16=True,                       # Better for modern GPUs
    dataloader_pin_memory=False,     # Save memory, Reduce memory pressure
    gradient_checkpointing=True,     # Trade compute for memory
    remove_unused_columns=False,     # Keep all columns including labels
    report_to=None,                  # Disable logging to avoid conflicts
    run_name=f"gemma-3-4b-resume-qlora-{batch_size}bs-{num_epochs}ep",
    seed=42,
    #torch_compile=False,             # Disable torch compile for compatibility
    #dataloader_num_workers=0,        # Single-threaded data loading for stability
)

In [20]:
print("Model loaded and prepared for fine-tuning.")
print(f"Starting training Argument...{training_args}")

Model loaded and prepared for fine-tuning.
Starting training Argument...TrainingArguments(
_n_gpu=1,
accelerator_config={'split_batches': False, 'dispatch_batches': None, 'even_batches': True, 'use_seedable_sampler': True, 'non_blocking': False, 'gradient_accumulation_kwargs': None, 'use_configured_state': False},
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
average_tokens_across_devices=True,
batch_eval_metrics=False,
bf16=True,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_persistent_workers=False,
dataloader_pin_memory=False,
dataloader_prefetch_factor=None,
ddp_backend=None,
ddp_broadcast_buffers=None,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_do_concat_batches=True,
eval_on_start=False,
eval_steps=5

In [21]:
from transformers import EarlyStoppingCallback
# Configure a callback to stop training if the evaluation loss
# doesn't improve for 3 consecutive evaluations.
early_stopping = EarlyStoppingCallback(
    early_stopping_patience=3,
    early_stopping_threshold=0.01
)

In [22]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    #tokenizer=tokenizer,
    data_collator=data_collator,
    callbacks=[early_stopping]
)


In [23]:
print(f"Started training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
trainer.train()

Started training at 2026-01-24 09:00:43


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice:

 2


wandb: You chose 'Use an existing W&B account'
wandb: Logging into https://api.wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: Find your API key here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: 2023ac05472 (2023ac05472-birla-institute-of-applied-sciences) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss,Validation Loss
50,0.068200,0.026473
100,0.011100,0.009994
150,0.008900,0.007420
200,0.006800,0.005941
250,0.004900,0.005531


TrainOutput(global_step=250, training_loss=0.13403101807832718, metrics={'train_runtime': 6408.2428, 'train_samples_per_second': 2.387, 'train_steps_per_second': 0.075, 'total_flos': 3.5639973314002944e+17, 'train_loss': 0.13403101807832718, 'epoch': 3.1253918495297803})

In [25]:
print(f"Completed training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"saving model at: {trained_model_path}")

Completed training at 2026-01-24 10:49:15
saving model at: models/fine_tuned_gemma-3-4b-it_resume_20260124-085837


In [26]:
model.save_pretrained(trained_model_path)
tokenizer.save_pretrained(trained_model_path)
print(f"Completed training at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

Completed training at 2026-01-24 10:49:22


In [ ]:
#from huggingface_hub import notebook_login
#notebook_login()

In [ ]:
import os, getpass

if 'HF_TOKEN' in os.environ:
    del os.environ['HF_TOKEN']

os.environ["HF_TOKEN"] = getpass.getpass("Enter your NEW Hugging Face WRITE token: ")
print(os.environ["HF_TOKEN"])


In [28]:
finetuned_model = model
finetuned_model_tokenizer=tokenizer

In [ ]:
repo_name = f"{fine_tuned_model_name}-v0.3"

# Push the model to the Hub
finetuned_model.push_to_hub(repo_name)

# Push the tokenizer to the Hub
finetuned_model_tokenizer.push_to_hub(repo_name)

## Model inferencing

In [31]:
print("Evaluating the model...")
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluating the model...


Evaluation results: {'eval_loss': 0.005531283561140299, 'eval_runtime': 85.8185, 'eval_samples_per_second': 5.244, 'eval_steps_per_second': 0.664, 'epoch': 3.1253918495297803}
